In [1]:
import os; os.chdir("../"); print(os.getcwd())

/Users/accor/Desktop/scripts/projects/mlops-nba


In [2]:
import pandas as pd
from mlops_nba.config import PRE_RAW_DATA_DIR, RAW_DATA_DIR
from mlops_nba.common.dates import convert_duration_to_number

In [3]:
data = pd.read_csv(PRE_RAW_DATA_DIR / "2024-01-17_2024-01-24"/ "boxscores.csv")
players = pd.read_csv(PRE_RAW_DATA_DIR / "2024-01-17_2024-01-24"/ "players.csv")

In [4]:
lebron = data[data["PLAYER_NAME"] == "LeBron James"]
lebron

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
671,22300605,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,31.000000:00,...,0.0,5.0,5.0,5.0,0.0,0.0,3.0,2.0,28.0,28.0
1064,22300591,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,33.000000:44,...,5.0,6.0,11.0,5.0,0.0,0.0,4.0,1.0,24.0,-13.0
1356,22300577,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,33.000000:03,...,1.0,7.0,8.0,8.0,1.0,0.0,3.0,1.0,25.0,20.0


In [5]:
# create and transform some columns
lebron["2PM"] = lebron["FGM"] - lebron["FG3M"]
lebron["2PA"] = lebron["FGA"] - lebron["FG3A"]
lebron["2P_PCT"] = lebron["2PM"] / lebron["2PA"]
lebron["mins"] = lebron["MIN"].apply(convert_duration_to_number)


/var/folders/vs/2y770xp508g1rrr52qxcdfz40000gp/T/ipykernel_54308/1501018612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lebron["2PM"] = lebron["FGM"] - lebron["FG3M"]
/var/folders/vs/2y770xp508g1rrr52qxcdfz40000gp/T/ipykernel_54308/1501018612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lebron["2PA"] = lebron["FGA"] - lebron["FG3A"]
/var/folders/vs/2y770xp508g1rrr52qxcdfz40000gp/T/ipykernel_54308/1501018612.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [6]:
lebron = lebron.merge(players, on="PLAYER_ID", how="left")
lebron

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME_x,START_POSITION,COMMENT,MIN,...,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,HOW_ACQUIRED
0,22300605,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,31.000000:00,...,lebron-james,23.0,F,6-9,250,"DEC 30, 1984",39.0,20,St. Vincent-St. Mary HS (OH),Signed on 07/09/18
1,22300591,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,33.000000:44,...,lebron-james,23.0,F,6-9,250,"DEC 30, 1984",39.0,20,St. Vincent-St. Mary HS (OH),Signed on 07/09/18
2,22300577,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,33.000000:03,...,lebron-james,23.0,F,6-9,250,"DEC 30, 1984",39.0,20,St. Vincent-St. Mary HS (OH),Signed on 07/09/18


In [7]:
aggregation_functions = {
    "first"  : ["TEAM_ABBREVIATION", "PLAYER_ID", "PLAYER_NAME", "AGE", "POSITION"],
    "mean"  : [
        'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT',
        "2PM", "2PA", "2P_PCT",
        'FTM', 'FTA', 'FT_PCT',
        'OREB', 'DREB',
        'REB', 'AST', 'STL',
        'BLK', 'TO', 'PF', 'PTS',
        'PLUS_MINUS',"mins"
    ],
    "count": ["GAME_ID", "START_POSITION"],
}

aggreations = {column:key for key,value in aggregation_functions.items() for column in value}


In [8]:
stats = lebron.groupby("PLAYER_NAME").agg(aggreations)
stats

,TEAM_ABBREVIATION,PLAYER_ID,PLAYER_NAME,AGE,POSITION,FGM,FGA,FG_PCT,FG3M,FG3A,...,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,mins,GAME_ID,START_POSITION
PLAYER_NAME,,,,,,,,,,,,,,,,,,,,,
LeBron James,LAL,2544,LeBron James,39.0,F,9.0,19.0,0.478333,2.666667,6.666667,...,6.0,0.333333,0.0,3.333333,1.333333,25.666667,11.666667,1955.666667,3,3


In [9]:
original = pd.read_csv(RAW_DATA_DIR / "first-drop"/"2023-2024 NBA Player Stats - Regular.csv", encoding="Windows-1252", sep=";")
original[original.Player == 'LeBron James']

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
219,216,LeBron James,PF,39,LAL,22,22,33.4,9.3,16.8,...,0.717,0.9,6.6,7.5,6.6,1.5,0.7,3.4,1.1,25.0


In [10]:
# 'eFG%', "Rk" --> not needed
mapping = {'TEAM_ABBREVIATION': 'Tm',
 'PLAYER_NAME': 'Player',
"POSITION":'Pos',
"AGE":'Age',
 'GAME_ID': 'G',
 'START_POSITION': 'GS',
 'mins': 'MP',
 'FGM': 'FG',
 'FGA': 'FGA',
 'FG_PCT': 'FG%',
 'FG3M': '3P',
 'FG3A': '3PA',
 'FG3_PCT': '3P%',
 '2PM': '2P',
 '2PA': '2PA',
 '2P_PCT': '2P%',
 'FTM': 'FT',
 'FTA': 'FTA',
 'FT_PCT': 'FT%',
 'OREB': 'ORB',
 'DREB': 'DRB',
 'REB': 'TRB',
 'AST': 'AST',
 'STL': 'STL',
 'BLK': 'BLK',
 'TO': 'TOV',
 'PF': 'PF',
 'PTS': 'PTS'
}

In [11]:
stats.rename(columns=mapping, inplace=True)
stats.to_dict(orient="records")

[{'Tm': 'LAL',
  'PLAYER_ID': 2544,
  'Player': 'LeBron James',
  'Age': 39.0,
  'Pos': 'F',
  'FG': 9.0,
  'FGA': 19.0,
  'FG%': 0.47833333333333333,
  '3P': 2.6666666666666665,
  '3PA': 6.666666666666667,
  '3P%': 0.4013333333333333,
  '2P': 6.333333333333333,
  '2PA': 12.333333333333334,
  '2P%': 0.5400326797385621,
  'FT': 5.0,
  'FTA': 5.666666666666667,
  'FT%': 0.9,
  'ORB': 2.0,
  'DRB': 6.0,
  'TRB': 8.0,
  'AST': 6.0,
  'STL': 0.3333333333333333,
  'BLK': 0.0,
  'TOV': 3.3333333333333335,
  'PF': 1.3333333333333333,
  'PTS': 25.666666666666668,
  'PLUS_MINUS': 11.666666666666666,
  'MP': 1955.6666666666667,
  'G': 3,
  'GS': 3}]

In [3]:
data = pd.read_csv(PRE_RAW_DATA_DIR / "2024-01-17_2024-01-24"/ "boxscores.csv")
players = pd.read_csv(PRE_RAW_DATA_DIR / "2024-01-17_2024-01-24"/ "players.csv")

In [9]:
data

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22300625,1610612756,PHX,Phoenix,1628960,Grayson Allen,Grayson,F,NaN,33.000000:37,...,1.0,4.0,5.0,4.0,1.0,0.0,2.0,3.0,15.0,26.0
1,22300625,1610612756,PHX,Phoenix,201142,Kevin Durant,Kevin,F,NaN,37.000000:39,...,0.0,10.0,10.0,7.0,1.0,2.0,1.0,4.0,12.0,19.0
2,22300625,1610612756,PHX,Phoenix,203994,Jusuf Nurkic,Jusuf,C,NaN,21.000000:08,...,3.0,2.0,5.0,3.0,2.0,0.0,4.0,4.0,8.0,23.0
3,22300625,1610612756,PHX,Phoenix,203078,Bradley Beal,Bradley,G,NaN,36.000000:10,...,2.0,5.0,7.0,7.0,1.0,1.0,2.0,4.0,20.0,9.0
4,22300625,1610612756,PHX,Phoenix,1626164,Devin Booker,Devin,G,NaN,34.000000:39,...,1.0,6.0,7.0,3.0,2.0,0.0,2.0,0.0,46.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1466,22300578,1610612757,POR,Portland,1629680,Matisse Thybulle,Matisse,NaN,NaN,20.000000:37,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,3.0,-3.0
1467,22300578,1610612757,POR,Portland,1631200,Kris Murray,Kris,NaN,NaN,2.000000:33,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0
1468,22300578,1610612757,POR,Portland,1629028,Deandre Ayton,Deandre,NaN,NWT - Not With Team,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1469,22300578,1610612757,POR,Portland,1631303,Justin Minaya,Justin,NaN,DNP - Coach's Decision,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
durant = data[data["PLAYER_NAME"] == "Deandre Ayton"]
durant

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
54,22300623,1610612757,POR,Portland,1629028,Deandre Ayton,Deandre,C,NaN,40.000000:44,...,6.0,11.0,17.0,3.0,1.0,2.0,1.0,3.0,18.0,15.0
262,22300617,1610612757,POR,Portland,1629028,Deandre Ayton,Deandre,C,NaN,27.000000:03,...,2.0,5.0,7.0,2.0,0.0,0.0,3.0,2.0,5.0,-9.0
659,22300605,1610612757,POR,Portland,1629028,Deandre Ayton,Deandre,C,NaN,22.000000:19,...,1.0,3.0,4.0,0.0,0.0,0.0,1.0,3.0,6.0,-17.0
962,22300590,1610612757,POR,Portland,1629028,Deandre Ayton,Deandre,C,NaN,30.000000:11,...,2.0,6.0,8.0,1.0,0.0,2.0,1.0,1.0,12.0,-14.0
1468,22300578,1610612757,POR,Portland,1629028,Deandre Ayton,Deandre,NaN,NWT - Not With Team,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
durant.merge(players, on="PLAYER_ID", how="left")

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME_x,START_POSITION,COMMENT,MIN,...,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,HOW_ACQUIRED
0,22300623,1610612757,POR,Portland,1629028,Deandre Ayton,Deandre,C,NaN,40.000000:44,...,deandre-ayton,2.0,C,7-0,247,"JUL 23, 1998",25.0,5,Arizona,Traded from PHX on 09/27/23
1,22300617,1610612757,POR,Portland,1629028,Deandre Ayton,Deandre,C,NaN,27.000000:03,...,deandre-ayton,2.0,C,7-0,247,"JUL 23, 1998",25.0,5,Arizona,Traded from PHX on 09/27/23
2,22300605,1610612757,POR,Portland,1629028,Deandre Ayton,Deandre,C,NaN,22.000000:19,...,deandre-ayton,2.0,C,7-0,247,"JUL 23, 1998",25.0,5,Arizona,Traded from PHX on 09/27/23
3,22300590,1610612757,POR,Portland,1629028,Deandre Ayton,Deandre,C,NaN,30.000000:11,...,deandre-ayton,2.0,C,7-0,247,"JUL 23, 1998",25.0,5,Arizona,Traded from PHX on 09/27/23
4,22300578,1610612757,POR,Portland,1629028,Deandre Ayton,Deandre,NaN,NWT - Not With Team,NaN,...,deandre-ayton,2.0,C,7-0,247,"JUL 23, 1998",25.0,5,Arizona,Traded from PHX on 09/27/23
